In [1]:
import yaml
from pathlib import Path

# Load params.yaml
params_path = Path("hparams.yaml")
with open(params_path, "r") as f:
    hparams = yaml.safe_load(f)

print(type(hparams))
print(hparams["batch_size"])
print(hparams["features"])

<class 'dict'>
500
['track', 'groundspeed', 'altitude', 'timedelta']


In [2]:
from sklearn.preprocessing import MinMaxScaler
from data.dataset import TrafficDataset

dataset_tcvae = TrafficDataset.from_file(
        "data/traffic_noga_tilFAF_train.pkl",
        features=["track", "groundspeed", "altitude", "timedelta"],
        scaler=MinMaxScaler(feature_range=(-1, 1)),
        shape= "image",
        info_params={"features": ["latitude", "longitude"], "index": -1},
    )

for sample in dataset_tcvae:
    break

len(sample), sample[0].shape,sample[1].shape

(2, torch.Size([4, 200]), torch.Size([2]))

In [3]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset_tcvae, batch_size=hparams["batch_size"], shuffle=True, num_workers=4)

for batch in loader:
    break

len(batch), batch[0].shape, batch[1].shape

(2, torch.Size([500, 4, 200]), torch.Size([500, 2]))

In [4]:
from tcn import TemporalBlock

t = TemporalBlock(
    in_channels=4,
    out_channels=hparams["h_dims"][-1],
    kernel_size=hparams["kernel_size"],
    dilation=hparams["dilation_base"],
    out_activ=None,
    dropout=0.2,
)

t

/Users/meldor/Desktop/vae-paper/.venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


TemporalBlock(
  (conv): Conv1d(4, 64, kernel_size=(16,), stride=(1,), dilation=(2,))
  (dropout): Dropout(p=0.2, inplace=False)
)

In [5]:
t(batch[0]).shape

torch.Size([500, 64, 200])

In [6]:
from tcn import ResidualBlock

r = ResidualBlock(
    in_channels=4,
    out_channels=hparams["h_dims"][-1],
    kernel_size=hparams["kernel_size"],
    dilation=hparams["dilation_base"],
    h_activ=None,
    dropout=0.2,
)

r

ResidualBlock(
  (tmp_block1): TemporalBlock(
    (conv): Conv1d(4, 64, kernel_size=(16,), stride=(1,), dilation=(2,))
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (tmp_block2): TemporalBlock(
    (conv): Conv1d(64, 64, kernel_size=(16,), stride=(1,), dilation=(2,))
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (downsample): Conv1d(4, 64, kernel_size=(1,), stride=(1,))
)

In [7]:
r(batch[0]).shape

torch.Size([500, 64, 200])

In [8]:
from tcn import TCN

tn = TCN(
    input_dim=4,
    out_dim=hparams["h_dims"][-1],
    h_dims=hparams["h_dims"],
    kernel_size=hparams["kernel_size"],
    dilation_base=hparams["dilation_base"],
    h_activ=None,
    dropout=0.2,)

tn

TCN(
  (network): Sequential(
    (0): ResidualBlock(
      (tmp_block1): TemporalBlock(
        (conv): Conv1d(4, 64, kernel_size=(16,), stride=(1,))
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (tmp_block2): TemporalBlock(
        (conv): Conv1d(64, 64, kernel_size=(16,), stride=(1,))
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (downsample): Conv1d(4, 64, kernel_size=(1,), stride=(1,))
    )
    (1): ResidualBlock(
      (tmp_block1): TemporalBlock(
        (conv): Conv1d(64, 64, kernel_size=(16,), stride=(1,), dilation=(2,))
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (tmp_block2): TemporalBlock(
        (conv): Conv1d(64, 64, kernel_size=(16,), stride=(1,), dilation=(2,))
        (dropout): Dropout(p=0.2, inplace=False)
      )
    )
    (2): ResidualBlock(
      (tmp_block1): TemporalBlock(
        (conv): Conv1d(64, 64, kernel_size=(16,), stride=(1,), dilation=(4,))
        (dropout): Dropout(p=0.2, inplace=False)
      )
  

In [9]:
tn(batch[0]).shape

torch.Size([500, 64, 200])

In [10]:
from encoder import Encoder

e = Encoder(
    input_dim=4,
    out_dim=hparams["h_dims"][-1],
    h_dims=hparams["h_dims"][:-1],
    kernel_size=hparams["kernel_size"],
    dilation_base=hparams["dilation_base"],
    sampling_factor=hparams["sampling_factor"],
    h_activ=None,
    dropout=0.2,
)

e

Encoder(
  (tcn): TCN(
    (network): Sequential(
      (0): ResidualBlock(
        (tmp_block1): TemporalBlock(
          (conv): Conv1d(4, 64, kernel_size=(16,), stride=(1,))
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (tmp_block2): TemporalBlock(
          (conv): Conv1d(64, 64, kernel_size=(16,), stride=(1,))
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (downsample): Conv1d(4, 64, kernel_size=(1,), stride=(1,))
      )
      (1): ResidualBlock(
        (tmp_block1): TemporalBlock(
          (conv): Conv1d(64, 64, kernel_size=(16,), stride=(1,), dilation=(2,))
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (tmp_block2): TemporalBlock(
          (conv): Conv1d(64, 64, kernel_size=(16,), stride=(1,), dilation=(2,))
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
      (2): ResidualBlock(
        (tmp_block1): TemporalBlock(
          (conv): Conv1d(64, 64, kernel_size=(16,), stride=(1,), dilati

In [11]:
#befoore flatten shape was 500x64x20
encoder_output = e(batch[0])
encoder_output.shape

torch.Size([500, 1280])

In [13]:
h_dim = hparams["h_dims"][-1] * (
            int(batch[0][0].shape[-1] / hparams["sampling_factor"])
        )
h_dim

1280

In [14]:
hparams["h_dims"][-1], batch[0][0].shape[-1], hparams["sampling_factor"]

(64, 200, 10)

In [15]:
from lsr import NormalLSR

lsr = NormalLSR(
    input_dim=h_dim,
    out_dim=hparams["encoding_dim"],
)

lsr(encoder_output), lsr(encoder_output).rsample().shape

(Independent(Normal(loc: torch.Size([500, 64]), scale: torch.Size([500, 64])), 1),
 torch.Size([500, 64]))

In [16]:
sampled = lsr(encoder_output).rsample()

In [17]:
batch[0][0].shape[0]

4

In [18]:
from lsr import VampPriorLSR

vamplsr = VampPriorLSR(
    original_dim=4,
    original_seq_len=200,
    input_dim=h_dim,
    out_dim=hparams["encoding_dim"],
    encoder=e,
    n_components=hparams["n_components"],  # Number of components in the VampPrior
)

vamplsr(encoder_output), vamplsr(encoder_output).rsample().shape

(Independent(Normal(loc: torch.Size([500, 64]), scale: torch.Size([500, 64])), 1),
 torch.Size([500, 64]))

In [19]:
sampled_vamp = vamplsr(encoder_output).rsample()

In [20]:
from decoder import TCDecoder
from torch import nn

d = decoder = TCDecoder(
            input_dim=hparams["encoding_dim"],
            out_dim=batch[0][0].shape[0],
            h_dims=hparams["h_dims"][::-1],
            seq_len=batch[0][0].shape[-1],
            kernel_size=hparams["kernel_size"],
            dilation_base=hparams["dilation_base"],
            sampling_factor=hparams["sampling_factor"],
            dropout=hparams["dropout"],
            h_activ=nn.ReLU(),
            # h_activ=None,
        )

reconstructed = d(sampled)
reconstructed.shape

torch.Size([500, 4, 200])

In [21]:
reconstructed_vamp = d(sampled_vamp)
reconstructed_vamp.shape

torch.Size([500, 4, 200])

In [22]:
from tcvae import TCVAE

tcv = TCVAE(hparams=hparams)
lsr_distparams, z, x_hat = tcv(batch[0])

In [23]:
x_hat.shape

torch.Size([500, 4, 200])

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

def get_device():
    if torch.backends.mps.is_available():
        return torch.device("mps")
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")

device = get_device()
print("Using device:", device)

Using device: mps


In [2]:
import yaml
from pathlib import Path

# Load params.yaml
params_path = Path("hparams.yaml")
with open(params_path, "r") as f:
    hparams = yaml.safe_load(f)

print(type(hparams))
print(hparams["batch_size"])
print(hparams["features"])

<class 'dict'>
500
['track', 'groundspeed', 'altitude', 'timedelta']


In [3]:
from sklearn.preprocessing import MinMaxScaler
from data.dataset import TrafficDataset

dataset_tcvae = TrafficDataset.from_file(
        "data/traffic_noga_tilFAF_train.pkl",
        features=["track", "groundspeed", "altitude", "timedelta"],
        scaler=MinMaxScaler(feature_range=(-1, 1)),
        shape= "image",
        info_params={"features": ["latitude", "longitude"], "index": -1},
    )

for sample in dataset_tcvae:
    break

len(sample), sample[0].shape,sample[1].shape

(2, torch.Size([4, 200]), torch.Size([2]))

In [4]:
loader = DataLoader(
    dataset_tcvae,
    batch_size=hparams["batch_size"],
    shuffle=True,
    num_workers=4,
    pin_memory=(device.type != "cpu"),
)

In [5]:
from tcvae import TCVAE

seq_len = dataset_tcvae.seq_len  # should be 200 given your config
model = TCVAE(hparams=hparams, inputdim=4, seq_length=seq_len).to(device)

/Users/meldor/Desktop/vae-paper/.venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [6]:
opt = torch.optim.Adam(model.parameters(), lr=hparams["lr"])

In [7]:
def empty_device_cache():
    if device.type == "cuda":
        torch.cuda.empty_cache()
        # torch.cuda.ipc_collect()  # optional
    elif device.type == "mps":
        torch.mps.empty_cache()

In [ ]:
from tqdm.auto import tqdm

model.train()

for epoch in range(500):
    pbar = tqdm(loader, desc=f"Epoch {epoch+1}/500", leave=False)
    running_loss = 0.0
    running_kld_loss = 0.0
    running_llv_loss = 0.0
    n_batches = 0

    for batch_idx, batch in enumerate(pbar):
        # Move batch to device
        x, y = batch
        x = x.to(device, non_blocking=True)
        # y is unused by the model, but keep it in the tuple structure
        batch = (x, y)

        opt.zero_grad(set_to_none=True)

        # Use the model's predefined training_step to compute ELBO loss
        elbo, kld_loss, llv_loss = model.training_step(batch, batch_idx)

        # Backprop + step
        elbo.backward()
        opt.step()

        # Track progress
        n_batches += 1
        running_loss += elbo.item()
        running_kld_loss += kld_loss
        running_llv_loss += llv_loss
        pbar.set_postfix({"train_loss": f"{running_loss / n_batches:.4f}"})

    # Optional: free device cache each epoch
    empty_device_cache()

    # Simple epoch print
    avg_loss = running_loss / max(1, n_batches)
    avg_kld_loss = running_kld_loss / max(1, n_batches)
    avg_llv_loss = running_llv_loss / max(1, n_batches)
    print(f"Epoch {epoch+1:03d}: train_loss={avg_loss:.6f}, kld_loss={avg_kld_loss:.6f}, llv_loss={avg_llv_loss:.6f}")

/Users/meldor/Desktop/vae-paper/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1/500:   0%|          | 0/28 [00:00<?, ?it/s]

Epoch 001: train_loss=789.793483, kld_loss=2.213984, llv_loss=787.579590


Epoch 2/500:   0%|          | 0/28 [00:00<?, ?it/s]

Epoch 002: train_loss=767.447486, kld_loss=4.271811, llv_loss=763.175720


...

Epoch 057: train_loss=748.032469, kld_loss=5.512986, llv_loss=742.519531


Epoch 58/500:   0%|          | 0/28 [00:00<?, ?it/s]

Epoch 058: train_loss=748.099418, kld_loss=5.466068, llv_loss=742.633240


Epoch 59/500:   0%|          | 0/28 [00:00<?, ?it/s]

/Users/meldor/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/multiprocessing/resource_tracker.py:213: UserWarning: ResourceTracker called reentrantly for resource cleanup, which is unsupported. The semaphore object '/loky-85692-zkhnrzhj' might leak.
  warnings.warn(
/Users/meldor/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/multiprocessing/resource_tracker.py:213: UserWarning: ResourceTracker called reentrantly for resource cleanup, which is unsupported. The semaphore object '/loky-85692-jrsvy8ve' might leak.
  warnings.warn(
/Users/meldor/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/multiprocessing/resource_tracker.py:213: UserWarning: ResourceTracker called reentrantly for resource cleanup, which is unsupported. The semaphore object '/loky-85692-rgtbwrv0' might leak.
  warnings.warn(
/Users/meldor/.local/share/uv/python/cpython-3.12.11-macos-aarch64-none/lib/python3.12/multiprocessing/resource_tracker.p

In [50]:
import torch
from torch.distributions import Normal, Independent

loc   = torch.tensor([[0., 0., 0.],    # sample 1 mean vector
                      [1., 1., 1.]])   # sample 2 mean vector
scale = torch.ones_like(loc)       # std = 1 everywhere

In [51]:
loc, scale

(tensor([[0., 0., 0.],
         [1., 1., 1.]]),
 tensor([[1., 1., 1.],
         [1., 1., 1.]]))

In [52]:
base = Normal(loc, scale)
z = torch.tensor([[0., 0., 0.],
                  [1., 2., 3.]])   # two latent vectors
logp = base.log_prob(z)
print(logp)

tensor([[-0.9189, -0.9189, -0.9189],
        [-0.9189, -1.4189, -2.9189]])


In [53]:
logp.sum(dim=-1)   # [2]

tensor([-2.7568, -5.2568])

In [48]:
diag_gauss = Independent(Normal(loc, scale), 1)
logp_indep = diag_gauss.log_prob(z)
print(logp_indep)

tensor([-2.7568, -5.2568])


In [49]:
diag_gauss.rsample()

tensor([[ 0.6864,  0.4128, -0.5349],
        [ 1.6668,  1.8176,  1.6142]])